In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore", "use_inf_as_na")
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras import models, Model
from tensorflow.keras.layers import LSTM, Input, GRU, SimpleRNN, Bidirectional, Dense, Dropout, Conv1D, MaxPooling1D, Flatten
from tensorflow.keras.utils import plot_model
from sklearn.metrics import mean_absolute_percentage_error, mean_absolute_error

In [ ]:
df = pd.read_csv('/kaggle/input/gold-price-10-years-20132023/Gold Price (2013-2023).csv')

In [ ]:
df.isna().mean()

In [ ]:
df.head()

## 1. EDA

### Preprocessing data

- Loại bỏ những dữ liệu thiếu do có ít dữ liệu thiếu

In [ ]:
df = df.dropna()

- Chuẩn hóa chuỗi sang số

In [ ]:
# Những feature cần chuẩn hóa sang số
num_cols = ['Price', 'Open', 'Low', 'High']

# Loại bỏ dấu , ở các feature số
df[num_cols] = df[num_cols].replace({',':''}, regex=True)

# Loại bỏ dấu , và ký tự K = 1000 ở feature Volumne
df['Vol.'] = df['Vol.'].replace({'K': '', '\.':''}, regex=True)

# Do feature 'Change %' là phần trăm thay đổi ngày hôm nay so với ngày trước đó 
df = df.drop(labels='Change %', axis=1)

- Chuyển đổi dữ liệu cột Date từ object sang datetime

In [ ]:
# Convert kiểu dữ liệu feature 'Date' từ object sang datetime
df['Date'] = pd.to_datetime(df['Date'])

# Sort values 
df.sort_values(by='Date', ascending=True, inplace=True)

# Reset lại index do có 1 số row bị drop
df.reset_index(drop=True, inplace=True)

- Chuyển đổi dữ liệu các cột số từ object float

In [ ]:
# Convert kiểu từ liệu các feature số từ object sang float
df[list(df.columns[1:])] = df[list(df.columns[1:])].astype('float64')

In [ ]:
df.dtypes

In [ ]:
df.head()

- Biểu đồ thể hiện sự thay đổi giá vàng từ năm 2013-2022

In [ ]:
fig, ax = plt.subplots()
sns.lineplot(data=df, x='Date', y='Price', ax=ax);
ax.set_title(label='Biểu đồ thể hiện sự thay đổi của giá vàng từ năm 2013-2023'.upper(), pad=20, size=15);
plt.xlabel('Năm');
plt.ylabel('Giá vàng');

- Sử dụng dữ liệu test năm 2022

In [ ]:
TEST_SIZE = df[df.Date.dt.year==2022].shape[0]
TEST_SIZE

In [ ]:
plt.figure(figsize=(15, 6), dpi=150)
plt.rcParams['axes.facecolor'] = 'white'
plt.rc('axes',edgecolor='white')
plt.plot(df.Date[:-TEST_SIZE], df.Price[:-TEST_SIZE], color='black', lw=2)
plt.plot(df.Date[-TEST_SIZE:], df.Price[-TEST_SIZE:], color='blue', lw=2)
plt.title('Gold Price Training and Test Sets', fontsize=15)
plt.xlabel('Date', fontsize=12)
plt.ylabel('Price', fontsize=12)
plt.legend(['Training set', 'Test set'], loc='upper left', prop={'size': 15})
plt.grid(color='white')
plt.show()

## 2. One Step Forecasting
- Từ số look back dự đoán 1 điểm dữ liệu trong tương lại (VD: Từ 14 ngày dự đoán ngày thứ 15 tiếp theo)

##### Chuẩn bị dữ liệu

In [ ]:
class OneStep:
    
    def __init__(self, data: pd.DataFrame, look_back=10):
        """
        Tham số:
            data (pd.DataFrame): Dữ liệu
            mapping_steps (int): Số lượng thời gian look back
        """
        # Lưu dữ liệu thành array thay vì DataFrame
        self.df = data.copy()
        self.data = self.df.values

        # Scaler dữ liệu
        self.scaler = MinMaxScaler()
        self.normalized_data = self.scaler.fit_transform(self.data)
        
        # Số lượng mẫu trong tập dữ liệu
        self.time_steps = data.shape[0] 
        
        # Số lượng đặc trưng
        self.n_columns = data.shape[1]

        # Số lượng ngày để dự đoán look_back -> output 
        self.look_back = look_back

    def data_mapping(self):
        """
        Chuyển từ 2D array sang 3D array để fit vào mô hình
        Với mỗi chuỗi có look_back thời gian tương ứng với số ngày trong bộ dữ liệu 
    
        Tham số:
            look_back (int): Số lượng thời gian cho từng chuỗi.
    
        Trả về:
            1 Array 3D với shape (mapping_iterations, look_back, n_columns).
        """
        mapping_steps = self.look_back + 1
        
        iterations = self.time_steps - mapping_steps + 1
        self.normalized_data_mapped = np.empty((iterations, mapping_steps, self.n_columns))
        
        for i in range(iterations):
            self.normalized_data_mapped[i, :, :] = self.normalized_data[i:i + mapping_steps, :]
        
        return self.normalized_data_mapped
    
    def train_test_split(self, test_size):
        """
        Chia tập dữ liệu thành tập train và test.
        
        Tham số:
            test_size (int): Số lượng quan sát thời gian cho tập test.
        
        Trả về:
            tuple: X_train, X_test, y_train, y_test
        """
        self.test_size = test_size
        self.train_size = self.normalized_data_mapped.shape[0] - self.test_size
        
        # normalized_data_mapped đã chứa số look_back+1 nên chỉ việc lấy look_back ở phần trước cho tập train và để lại giá trị cuối cùng là của tập test
        X_train = self.normalized_data_mapped[:self.train_size, :-1, :]
        y_train = self.normalized_data_mapped[:self.train_size, -1, :]
        
        X_test = self.normalized_data_mapped[self.train_size:, :-1, :]
        y_test = self.normalized_data_mapped[self.train_size:, -1, :]
        
        return X_train, X_test, y_train, y_test  

    def forecast_n_steps(self, model, data: pd.DataFrame, n_forecast_steps=30):
        """
        Dự đoán nhiều điểm dữ liệu tiếp theo bằng cách từ model dự đoán 1 step. Bỏ dữ liệu ban đầu từ chuỗi look back sau đó đưa dữ liệu vừa dự đoán
        vào cuối của chuỗi look back để dự đoán tiếp theo. Lặp lại cho đến khi dự đoán được đủ n_forecast_steps
    
        Tham số:
            model: Mô hình để dự đoán.
            data: 1 chuỗi look back bất kì để phán đoán step tiếp theo của look back.
            n_forecast_steps: Số lượng step được dự đoán tiếp sau chuỗi look back.
    
        Trả về:
            np.array: 1 tập n_forecast_steps đã dự đoán.
        """
        # Scaling dữ liệu của chuỗi look back. last_steps shape = (self.look_back, self.n_columns)
        last_steps = self.scaler.transform(data.values)[-self.look_back:]
    
        # Định nghĩa 1 mẫu dữ liệu để dữ đoán output với shape = (n_forecast_steps, self.look_back, self.n_columns) để có thể fit từng mẫu vào mô hình dự đoán ra output
        normalized_data_mapped = np.empty((n_forecast_steps, self.look_back, self.n_columns))
    
        # Khởi tạo tập dự đoán chứa n_forecast_steps dự đoán từ mô hình
        predictions = np.empty((n_forecast_steps, self.n_columns))
    
        # Dự đoán trước 1 step
        normalized_data_mapped[0, :, :] = last_steps
        predictions[0, :] = model.predict(
            normalized_data_mapped[0, :, :].reshape(1, self.look_back, self.n_columns),
            verbose=False
        )
        # Tạo ra n_forecast_steps dự đoán mới từ look back và 1 step đã dự đoán
        for i in range(1, n_forecast_steps):
            # Loại bỏ quan sát ở đầu ra và thêm phán đoán mới vào chuỗi
            normalized_data_mapped[i, :-1, :] = normalized_data_mapped[i - 1, 1:, :]
            normalized_data_mapped[i, -1, :] = predictions[i - 1, :]
    
            # Dự đoán quan sát tiếp theo
            norm_data = normalized_data_mapped[i, :, :].reshape(1, self.look_back, self.n_columns)
            predictions[i, :] = model.predict(norm_data, verbose=False)
    
        # Inverse dữ liệu 
        predictions = self.scaler.inverse_transform(predictions)
        return predictions

- Các đặc trưng dùng để training

In [ ]:
columns = ['Price', 'Open', 'Low', 'High', 'Vol.']

In [ ]:
# Từ 30 điểm thời gian trước đó dự đoán ra điểm thời gian thứ 31 tiếp theo
look_back = 30

# Fit dữ liệu để setup dữ liệu cho mô hình
data_model_one_step = OneStep(df[columns], look_back)

# Scaled dữ liệu
scaled_data_mapped = data_model_one_step.data_mapping()

- test_size = 256: tương ứng với giá vàng năm 2022

In [ ]:
X_train_one_step, X_test_one_step, y_train_one_step, y_test_one_step = data_model_one_step.train_test_split(test_size=TEST_SIZE)

In [ ]:
X_train_one_step.shape, X_test_one_step.shape, y_train_one_step.shape, y_test_one_step.shape

In [ ]:
# Lưu lại kết quả
import random
random.seed(0)
np.random.seed(0)
tf.random.set_seed(0)

#### 2.1 Mô hình LSTM one step

##### Định nghĩa mô hình

In [ ]:
def lstm_one_step_model():
    input = Input(shape=X_train_one_step[0].shape)
    x = LSTM(units=128, return_sequences=True)(input)
    x = Dropout(0.2)(x)
    x = LSTM(units=64, return_sequences=False)(x)
    x = Dense(units=32, activation='relu')(x)
    output = Dense(units=len(columns))(x)
    
    model = Model(inputs=input, outputs=[output])
    model.compile(loss='mean_squared_error', optimizer='adam')
    model.summary()
    
    return model

In [ ]:
model1 = lstm_one_step_model()

In [ ]:
plot_model(model1, show_shapes=True, expand_nested=True, dpi=50)

##### Đưa dữ liệu vào mô hình học
- Lựa chọn tham số
    + epochs = 50, số lần học toàn bộ tập train
    + batch_size = 64, mỗi iteration sẽ training 64 mẫu
    + validation_split = 0.1, lấy 10% số dự liệu tập train làm dữ liệu validate

In [ ]:
history1 = model1.fit(X_train_one_step, y_train_one_step, epochs=50, batch_size=128, validation_split=0.1, verbose=1)

- Đánh giá tốc độ học của mô hình

In [ ]:
plt.plot(history1.history['loss'], label = 'training loss')
plt.plot(history1.history['val_loss'], label ='validation loss')
plt.legend()
plt.show()

- Đường màu xanh cho thấy mô hình đã học khá nhanh sau 1 vài epochs đầu
- Đường màu cam giảm thấp hơn tức là mô hình có mức độ khái quát cao và không bị overfitting

- Đánh giá mô hình trên tập test và phán đoán trên tập test

In [ ]:
result1 = model1.evaluate(X_test_one_step, y_test_one_step)
y_pred_model1 = model1.predict(X_test_one_step)

- Sử dụng metric đánh giá mô hình
    + MAPE: phần trăm sai số giữa giá trị dự báo và giá trị thực tế 

In [ ]:
y_test_true_one_step = data_model_one_step.scaler.inverse_transform(y_test_one_step).T[0]
y_test_pred_model1 = data_model_one_step.scaler.inverse_transform(y_pred_model1).T[0]

In [ ]:
MAPE = mean_absolute_percentage_error(y_test_true_one_step, y_test_pred_model1)
Accuracy = 1 - MAPE
print(f"Phần trăm sai số giữa giá trị dự báo và giá trị thực tế {MAPE * 100}")
print(f"Độ chính xác của mô hình: {Accuracy * 100}")

In [ ]:
plt.figure(figsize=(15, 6), dpi=150)
plt.rcParams['axes.facecolor'] = 'white'
plt.rc('axes',edgecolor='white')
plt.plot(df['Date'].iloc[:-TEST_SIZE], df['Price'].iloc[:-TEST_SIZE], color='black', lw=2)
plt.plot(df['Date'].iloc[-TEST_SIZE:], y_test_true_one_step, color='blue', lw=2)
plt.plot(df['Date'].iloc[-TEST_SIZE:], y_test_pred_model1, color='red', lw=2)
plt.title('Biểu đồ thể hiện giá vàng phán đoán của mô hình LSTM', fontsize=15)
plt.xlabel('Ngày', fontsize=12)
plt.ylabel('Giá vàng', fontsize=12)
plt.legend(['Training Data', 'Actual Test Data', 'Predicted Test Data'], loc='upper left', prop={'size': 15})
plt.grid(color='white')
plt.show()

In [ ]:
print(f"Biên độ giao động của giá trị dự báo xung quanh giá trị thực tế là {((y_test_true_one_step - y_test_pred_model1) **2).mean()**(1/2)}")

#### Dự đoán n giá trị trong tương lai từ mô hình LSTM one step

- Dự đoán 15 ngày tiếp theo trong tương lai từ mô hình bằng cách lấy các giá trị phán đoán của mô hình đưa vào chuỗi phán đoán tiếp tục đến khi đủ 15 ngày

In [ ]:
n_steps = 15

In [ ]:
n_steps_data = df[columns].iloc[-(look_back + n_steps): -n_steps, :]

In [ ]:
n_steps_prediction_model1 = data_model_one_step.forecast_n_steps(model1, n_steps_data, n_steps)

In [ ]:
plt.figure(figsize=(15, 6), dpi=150)
plt.rcParams['axes.facecolor'] = 'white'
plt.rc('axes',edgecolor='white')
plt.plot(df['Date'].iloc[-(look_back + n_steps): -n_steps], df['Price'].iloc[-(look_back + n_steps): -n_steps], color='black', lw=2)
plt.plot(df['Date'].iloc[-n_steps:], df['Price'].iloc[-n_steps:], color='blue', lw=2)
plt.plot(df['Date'].iloc[-n_steps:], n_steps_prediction_model1.T[0], color='red', lw=2)
plt.title(f'Biểu đồ thể hiện phán đoán của mô hình LSTM {n_steps} ngày liên tiếp trong tương lai', fontsize=15)
plt.xlabel('Ngày', fontsize=12)
plt.ylabel('Giá vàng', fontsize=12)
plt.legend(['Actual data', 'Unseen data', 'Predicted Data'], loc='upper left', prop={'size': 15})
plt.grid(color='white')
plt.show()

In [ ]:
print(f"Biên độ giao động của giá trị dự báo xung quanh giá trị thực tế là {((df['Price'].iloc[-n_steps:].values - n_steps_prediction_model1.T[0]) **2).mean()**(1/2)}")

- Dựa vào biên độ giao động có thể thấy mô hình đưa ra các phán đoán cho các giá trị thời gian tương lai xa hơn 1 bị sai lệch rất nhiều vào đường dự đoán (đường màu đỏ theo biểu đồ) gần như là đường tuyến tính

#### 2.2. Mô hình GRU one step

##### Định nghĩa mô hình

In [ ]:
def gru_one_step_model():
    input = Input(shape=X_train_one_step[0].shape)
    x = GRU(units=64, return_sequences=False)(input)
    x = Dropout(0.1)(x)
    x = Dense(units=32, activation='relu')(x)
    x = Dropout(0.1)(x)
    output = Dense(units=len(columns))(x)
    
    model = Model(inputs=input, outputs=[output])
    model.compile(loss='mean_squared_error', optimizer='adam')
    model.summary()
    
    return model

In [ ]:
model2 = gru_one_step_model()

##### Đưa dữ liệu vào mô hình học
- Lựa chọn tham số
    + epochs = 50, số lần học toàn bộ tập train
    + batch_size = 64, mỗi iteration sẽ training 64 mẫu
    + validation_split = 0.1, lấy 10% số dự liệu tập train làm dữ liệu validate

In [ ]:
history2 = model2.fit(X_train_one_step, y_train_one_step, epochs=50, batch_size=64, validation_split=0.1, verbose=1)

- Đánh giá tốc độ học của mô hình

In [ ]:
plt.plot(history2.history['loss'], label = 'training loss')
plt.plot(history2.history['val_loss'], label ='validation loss')
plt.legend()
plt.show()

- Đường màu xanh cho thấy mô hình đã học khá nhanh sau 1 vài epochs đầu
- Đường màu cam giảm thấp hơn tức là mô hình có mức độ khái quát cao và không bị overfitting

- Đánh giá mô hình trên tập test và phán đoán trên tập test

In [ ]:
result2 = model2.evaluate(X_test_one_step, y_test_one_step)
y_pred_model2 = model2.predict(X_test_one_step)

- Sử dụng metric đánh giá mô hình
    + MAPE: phần trăm sai số giữa giá trị dự báo và giá trị thực tế 

In [ ]:
y_test_pred_model2 = data_model_one_step.scaler.inverse_transform(y_pred_model2).T[0]
MAPE = mean_absolute_percentage_error(y_test_true_one_step, y_test_pred_model2)
Accuracy = 1 - MAPE
print(f"Phần trăm sai số giữa giá trị dự báo và giá trị thực tế {MAPE * 100}")
print(f"Độ chính xác của mô hình: {Accuracy * 100}")

In [ ]:
plt.figure(figsize=(15, 6), dpi=150)
plt.rcParams['axes.facecolor'] = 'white'
plt.rc('axes',edgecolor='white')
plt.plot(df['Date'].iloc[:-TEST_SIZE], df['Price'].iloc[:-TEST_SIZE], color='black', lw=2)
plt.plot(df['Date'].iloc[-TEST_SIZE:], y_test_true_one_step, color='blue', lw=2)
plt.plot(df['Date'].iloc[-TEST_SIZE:], y_test_pred_model2, color='red', lw=2)
plt.title('Biểu đồ thể hiện giá vàng phán đoán của mô hình GRU', fontsize=15)
plt.xlabel('Ngày', fontsize=12)
plt.ylabel('Giá vàng', fontsize=12)
plt.legend(['Training Data', 'Actual Test Data', 'Predicted Test Data'], loc='upper left', prop={'size': 15})
plt.grid(color='white')
plt.show()

In [ ]:
print(f"Biên độ giao động của giá trị dự báo xung quanh giá trị thực tế là {((y_test_true_one_step - y_test_pred_model2) **2).mean()**(1/2)}")

#### Dự đoán n giá trị trong tương lai từ mô hình LSTM one step
- n_steps = 15

In [ ]:
n_steps_prediction_model2 = data_model_one_step.forecast_n_steps(model2, n_steps_data, n_steps)

In [ ]:
plt.figure(figsize=(15, 6), dpi=150)
plt.rcParams['axes.facecolor'] = 'white'
plt.rc('axes',edgecolor='white')
plt.plot(df['Date'].iloc[-(look_back + n_steps): -n_steps], df['Price'].iloc[-(look_back + n_steps): -n_steps], color='black', lw=2)
plt.plot(df['Date'].iloc[-n_steps:], df['Price'].iloc[-n_steps:], color='blue', lw=2)
plt.plot(df['Date'].iloc[-n_steps:], n_steps_prediction_model2.T[0], color='red', lw=2)
plt.title(f'Biểu đồ thể hiện phán đoán của mô hình GRU {n_steps} ngày liên tiếp trong tương lai', fontsize=15)
plt.xlabel('Ngày', fontsize=12)
plt.ylabel('Giá vàng', fontsize=12)
plt.legend(['Actual data', 'Unseen data', 'Predicted Data'], loc='upper left', prop={'size': 15})
plt.grid(color='white')
plt.show()

- Dựa vào biên độ giao động có thể thấy mô hình đưa ra các phán đoán cho các giá trị thời gian tương lai xa hơn 1 bị sai lệch rất nhiều vào đường dự đoán (đường màu đỏ theo biểu đồ) gần như là đường tuyến tính

In [ ]:
print(f"Biên độ giao động của giá trị dự báo xung quanh giá trị thực tế là {((df['Price'].iloc[-n_steps:].values - n_steps_prediction_model2.T[0]) **2).mean()**(1/2)}")

#### 2.3. Mô hình BiLSTM one step

##### Định nghĩa mô hình

In [ ]:
def bilstm_one_step_model():
    input = Input(shape=X_train_one_step[0].shape)
    x = Bidirectional(LSTM(units=64, return_sequences=False))(input)
    x = Dropout(0.1)(x)
    x = Dense(units=32, activation='relu')(x)
    x = Dropout(0.1)(x)
    output = Dense(units=len(columns))(x)
    
    model = Model(inputs=input, outputs=[output])
    model.compile(loss='mean_squared_error', optimizer='adam')
    model.summary()
    
    return model

In [ ]:
model3 = bilstm_one_step_model()

##### Đưa dữ liệu vào mô hình học
- Lựa chọn tham số
    + epochs = 50, số lần học toàn bộ tập train
    + batch_size = 64, mỗi iteration sẽ training 64 mẫu
    + validation_split = 0.1, lấy 10% số dự liệu tập train làm dữ liệu validate

In [ ]:
history3 = model3.fit(X_train_one_step, y_train_one_step, epochs=50, batch_size=64, validation_split=0.1, verbose=1)

- Đánh giá tốc độ học của mô hình

In [ ]:
plt.plot(history3.history['loss'], label = 'training loss')
plt.plot(history3.history['val_loss'], label ='validation loss')
plt.legend()
plt.show()

- Đánh giá mô hình trên tập test và phán đoán trên tập test

In [ ]:
result3 = model3.evaluate(X_test_one_step, y_test_one_step)
y_pred_model3 = model3.predict(X_test_one_step)

- Sử dụng metric đánh giá mô hình
    + MAPE: phần trăm sai số giữa giá trị dự báo và giá trị thực tế 

In [ ]:
y_test_pred_model3 = data_model_one_step.scaler.inverse_transform(y_pred_model3).T[0]
MAPE = mean_absolute_percentage_error(y_test_true_one_step, y_test_pred_model3)
Accuracy = 1 - MAPE
print(f"Phần trăm sai số giữa giá trị dự báo và giá trị thực tế {MAPE * 100}")
print(f"Độ chính xác của mô hình: {Accuracy * 100}")

In [ ]:
plt.figure(figsize=(15, 6), dpi=150)
plt.rcParams['axes.facecolor'] = 'white'
plt.rc('axes',edgecolor='white')
plt.plot(df['Date'].iloc[:-TEST_SIZE], df['Price'].iloc[:-TEST_SIZE], color='black', lw=2)
plt.plot(df['Date'].iloc[-TEST_SIZE:], y_test_true_one_step, color='blue', lw=2)
plt.plot(df['Date'].iloc[-TEST_SIZE:], y_test_pred_model3, color='red', lw=2)
plt.title('Biểu đồ thể hiện giá vàng phán đoán của mô hình BiLSTM', fontsize=15)
plt.xlabel('Ngày', fontsize=12)
plt.ylabel('Giá vàng', fontsize=12)
plt.legend(['Training Data', 'Actual Test Data', 'Predicted Test Data'], loc='upper left', prop={'size': 15})
plt.grid(color='white')
plt.show()

In [ ]:
print(f"Biên độ giao động của giá trị dự báo xung quanh giá trị thực tế là {((y_test_true_one_step - y_test_pred_model3) **2).mean()**(1/2)}")

#### Dự đoán n giá trị trong tương lai từ mô hình BiLSTM one step
- n_steps = 15

In [ ]:
n_steps_prediction_model3 = data_model_one_step.forecast_n_steps(model3, n_steps_data, n_steps)

In [ ]:
plt.figure(figsize=(15, 6), dpi=150)
plt.rcParams['axes.facecolor'] = 'white'
plt.rc('axes',edgecolor='white')
plt.plot(df['Date'].iloc[-(look_back + n_steps): -n_steps], df['Price'].iloc[-(look_back + n_steps): -n_steps], color='black', lw=2)
plt.plot(df['Date'].iloc[-n_steps:], df['Price'].iloc[-n_steps:], color='blue', lw=2)
plt.plot(df['Date'].iloc[-n_steps:], n_steps_prediction_model3.T[0], color='red', lw=2)
plt.title(f'Biểu đồ thể hiện phán đoán của mô hình BiLSTM {n_steps} ngày liên tiếp trong tương lai', fontsize=15)
plt.xlabel('Ngày', fontsize=12)
plt.ylabel('Giá vàng', fontsize=12)
plt.legend(['Actual data', 'Unseen data', 'Predicted Data'], loc='upper left', prop={'size': 15})
plt.grid(color='white')
plt.show()

In [ ]:
print(f"Biên độ giao động của giá trị dự báo xung quanh giá trị thực tế là {((df['Price'].iloc[-n_steps:].values - n_steps_prediction_model3.T[0]) **2).mean()**(1/2)}")

#### 2.4. Mô hình CNN one step

##### Định nghĩa mô hình

In [ ]:
def cnn_one_step_model():
    input = Input(shape=X_train_one_step[0].shape)
    x = Conv1D(filters=64, kernel_size=3)(input)
    x = MaxPooling1D(pool_size=3)(x)
    x = Flatten()(x)
    x = Dense(units=64, activation='relu')(x)
    x = Dropout(0.1)(x)
    output = Dense(units=len(columns))(x)
    
    model = Model(inputs=input, outputs=[output])
    model.compile(loss='mean_squared_error', optimizer='adam')
    model.summary()
    
    return model

In [ ]:
model4 = cnn_one_step_model()

##### Đưa dữ liệu vào mô hình học
- Lựa chọn tham số
    + epochs = 50, số lần học toàn bộ tập train
    + batch_size = 64, mỗi iteration sẽ training 64 mẫu
    + validation_split = 0.1, lấy 10% số dự liệu tập train làm dữ liệu validate

In [ ]:
history4 = model4.fit(X_train_one_step, y_train_one_step, epochs=50, batch_size=64, validation_split=0.1, verbose=1)

- Đánh giá tốc độ học của mô hình

In [ ]:
plt.plot(history4.history['loss'], label = 'training loss')
plt.plot(history4.history['val_loss'], label ='validation loss')
plt.legend()
plt.show()

- Đánh giá mô hình trên tập test và phán đoán trên tập test

In [ ]:
result4 = model4.evaluate(X_test_one_step, y_test_one_step)
y_pred_model4 = model4.predict(X_test_one_step)

- Sử dụng metric đánh giá mô hình
    + MAPE: phần trăm sai số giữa giá trị dự báo và giá trị thực tế 

In [ ]:
y_test_pred_model4 = data_model_one_step.scaler.inverse_transform(y_pred_model4).T[0]
MAPE = mean_absolute_percentage_error(y_test_true_one_step, y_test_pred_model4)
Accuracy = 1 - MAPE
print(f"Phần trăm sai số giữa giá trị dự báo và giá trị thực tế {MAPE * 100}")
print(f"Độ chính xác của mô hình: {Accuracy * 100}")

In [ ]:
plt.figure(figsize=(15, 6), dpi=150)
plt.rcParams['axes.facecolor'] = 'white'
plt.rc('axes',edgecolor='white')
plt.plot(df['Date'].iloc[:-TEST_SIZE], df['Price'].iloc[:-TEST_SIZE], color='black', lw=2)
plt.plot(df['Date'].iloc[-TEST_SIZE:], y_test_true_one_step, color='blue', lw=2)
plt.plot(df['Date'].iloc[-TEST_SIZE:], y_test_pred_model4, color='red', lw=2)
plt.title('Biểu đồ thể hiện giá vàng phán đoán của mô hình CNN', fontsize=15)
plt.xlabel('Ngày', fontsize=12)
plt.ylabel('Giá vàng', fontsize=12)
plt.legend(['Training Data', 'Actual Test Data', 'Predicted Test Data'], loc='upper left', prop={'size': 15})
plt.grid(color='white')
plt.show()

In [ ]:
print(f"Biên độ giao động của giá trị dự báo xung quanh giá trị thực tế là {((y_test_true_one_step - y_test_pred_model4) **2).mean()**(1/2)}")

#### Dự đoán n giá trị trong tương lai từ mô hình CNN one step
- n_steps = 15

In [ ]:
n_steps_prediction_model4 = data_model_one_step.forecast_n_steps(model4, n_steps_data, n_steps)

In [ ]:
plt.figure(figsize=(15, 6), dpi=150)
plt.rcParams['axes.facecolor'] = 'white'
plt.rc('axes',edgecolor='white')
plt.plot(df['Date'].iloc[-(look_back + n_steps): -n_steps], df['Price'].iloc[-(look_back + n_steps): -n_steps], color='black', lw=2)
plt.plot(df['Date'].iloc[-n_steps:], df['Price'].iloc[-n_steps:], color='blue', lw=2)
plt.plot(df['Date'].iloc[-n_steps:], n_steps_prediction_model4.T[0], color='red', lw=2)
plt.title(f'Biểu đồ thể hiện phán đoán của mô hình BiLSTM {n_steps} ngày liên tiếp trong tương lai', fontsize=15)
plt.xlabel('Ngày', fontsize=12)
plt.ylabel('Giá vàng', fontsize=12)
plt.legend(['Actual data', 'Unseen data', 'Predicted Data'], loc='upper left', prop={'size': 15})
plt.grid(color='white')
plt.show()

In [ ]:
print(f"Biên độ giao động của giá trị dự báo xung quanh giá trị thực tế là {((df['Price'].iloc[-n_steps:].values - n_steps_prediction_model4.T[0]) **2).mean()**(1/2)}")

## 3. Multi Step Forecasting
- Từ n_steps_in dự đoán n_steps_out tương lai (VD: Từ 30 ngày trong quá khứ dự đoán 14 ngày tiếp theo)

In [ ]:
class MultiStep:
    def __init__(self, data, n_steps_in, n_steps_out):
        """
        Khởi tạo dữ liệu từ data truyền vào và biến đổi dữ liệu
        
        Tham số:
            data: dữ liệu truyền vào
            n_steps_in: Số lượng quan sát thời gian cho mỗi chuỗi đầu vào
            n_steps_out: Số lượng quan sát thời gian cho mỗi chuỗi đầu ra
        """
        self.df = data.copy()
        self.n_steps_in = n_steps_in
        self.n_steps_out = n_steps_out
        self.n_columns = self.df.shape[1]
        
        self.scaler = MinMaxScaler()
        self.normalized_data = self.scaler.fit_transform(self.df.values)

    def data_mapping(self):
        """
        Từ dữ liệu đầu vào 2D array biến đổi thành các chuỗi quan sát thời gian với mỗi chuỗi input gồm n_steps_in quan sát thời gian
        và mỗi chuỗi output gồm n_steps_out quan sát thời gian
        Chuỗi chia với sải bước là 1
        
        Trả về:
            X: 3D array chuỗi input
            y: 3D array chuỗi output
        """
        num_samples = len(self.normalized_data) - self.n_steps_in - self.n_steps_out + 1
        
        X = np.empty((num_samples, self.n_steps_in, self.n_columns))
        y = np.empty((num_samples, self.n_steps_out, self.n_columns))

        for i in range(num_samples):
            X[i, :, :] = self.normalized_data[i:i + self.n_steps_in, :]
            y[i, :, :] = self.normalized_data[i + self.n_steps_in:i + self.n_steps_in + self.n_steps_out, :]

        return X, y

    def train_test_split(self, X, y, test_size):
        """
        Chia dữ liệu đã được ánh xạ sang các chuỗi thành tập train và test
        
        Tham số:
            X: input
            y: output
            test_size: Kích thước tập test
        
        Trả về:
            X_train, X_test, y_train, y_test
        """
        X_train, y_train = X[:-test_size], y[:-test_size]
        X_test, y_test = X[-test_size:], y[-test_size:]

        return X_train, X_test, y_train, y_test

    def multi_step_forecast(self, model, data: pd.DataFrame, iterations=1):
        """
        Dự đoán nhiều điểm dữ liệu tiếp theo bằng cách từ model dự đoán n_step_out. Bỏ dữ liệu ban đầu từ chuỗi n_steps_in sau đó đưa dữ liệu vừa dự đoán
        vào cuối của chuỗi n_steps_in để dự đoán tiếp theo. Lặp lại cho đến khi dự đoán được đủ iterations
    
        Tham số:
            model: Mô hình để dự đoán.
            data: 1 chuỗi n_steps_in bất kì để phán đoán n_step_out tiếp theo
            iterations: Số lần n_steps_out
    
        Trả về:
            np.array: 1 tập n_forecast_steps đã dự đoán.
        """
        # Scaling dữ liệu của chuỗi look back. last_steps shape = (self.n_steps_in, self.n_columns)
        last_steps = self.scaler.transform(data.values)
    
        # Định nghĩa 1 mẫu dữ liệu để dữ đoán output với shape = (n_forecast_steps, self.look_back, self.n_columns) để có thể fit từng mẫu vào mô hình dự đoán ra output
        normalized_data_mapped = np.empty((iterations, self.n_steps_in, self.n_columns))
    
        # Khởi tạo tập dự đoán chứa n_forecast_steps dự đoán từ mô hình
        predictions = np.empty((iterations, self.n_steps_out * self.n_columns))
    
        # Dự đoán trước 1 step
        normalized_data_mapped[0, :, :] = last_steps
        predictions[0, :] = model.predict(
            normalized_data_mapped[0, :, :].reshape(1, self.n_steps_in, self.n_columns),
            verbose=False
        )
        
        # Tạo ra n_forecast_steps dự đoán mới từ look back và 1 step đã dự đoán
        for i in range(1, iterations):
            # Loại bỏ quan sát ở đầu ra và thêm phán đoán mới vào chuỗi
            normalized_data_mapped[i, :-self.n_steps_out, :] = normalized_data_mapped[i - 1, self.n_steps_out:, :]
            normalized_data_mapped[i, -self.n_steps_out:, :] = predictions[i - 1, :].reshape(self.n_steps_out, self.n_columns)
    
            # Dự đoán quan sát tiếp theo
            norm_data = normalized_data_mapped[i, :, :].reshape(1, self.n_steps_in, self.n_columns)
            predictions[i, :] = model.predict(norm_data, verbose=False)
    
        # Inverse dữ liệu 
        predictions = predictions.reshape(-1, self.n_steps_out, self.n_columns)
        
        forecast_data = []
        
        for i in range(iterations):
            predictions[i] = self.scaler.inverse_transform(predictions[i])
            forecast_data += list(predictions[i].T[0])
        
        return forecast_data

- Các đặc trưng sử dụng

In [ ]:
columns = ['Price', 'Open', 'Low', 'High', 'Vol.']

- Lựa chọn 90 quan sát thời gian dự đoán 15 quan sát thời gian

In [ ]:
n_steps_in = 90
n_steps_out = 10

In [ ]:
# Đưa dữ liệu vào để biến đổi 
msp_data_model = MultiStep(df[columns], n_steps_in, n_steps_out)

# data đã được scaled và biến đổi theo 90 ngày dự đoán 14 ngày
X, y = msp_data_model.data_mapping()

In [ ]:
X.shape, y.shape

In [ ]:
X_train, X_test, y_train, y_test = msp_data_model.train_test_split(X, y, TEST_SIZE)

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

#### 3.1. Dự đoán multistep bằng mô hình LSTM

In [ ]:
def lstm_multi_step_model():
    input = Input(shape=X_train[0].shape)
    x = LSTM(units=64, return_sequences=False)(input)
    x = Dropout(0.1)(x)
    x = Dense(32, activation='relu')(x)
    output = Dense(n_steps_out*len(columns))(x)

    model = Model(inputs=input, outputs=[output])
    model.compile(loss='mean_squared_error', optimizer='adam')
    model.summary()
    
    return model

In [ ]:
model5 = lstm_multi_step_model()

In [ ]:
history5 = model5.fit(X_train, y_train.reshape(-1, n_steps_out*len(columns)), epochs=50, batch_size=128, validation_split=0.1, verbose=1)

In [ ]:
result5 = model5.evaluate(X_test, y_test.reshape(-1, n_steps_out*len(columns)))
y_test_pred_model5 = model5.predict(X_test)

- Lấy ra giá vàng của tập test để đánh giá

In [ ]:
y_test_pred_model5 = y_test_pred_model5.reshape(-1, n_steps_out, len(columns))
iterations = TEST_SIZE // n_steps_out + 1
remain = TEST_SIZE % n_steps_out

y_price_test_pred_model5 = []

for i in range(iterations):
    y_price_test_pred_model5 += list(msp_data_model.scaler.inverse_transform(y_test_pred_model5[i*n_steps_out]).T[0])
    
if remain != 0:
    y_price_test_pred_model5 += list(msp_data_model.scaler.inverse_transform(y_test_pred_model5[-1]).T[0][-remain+1:])

In [ ]:
plt.figure(figsize=(15, 6), dpi=150)
plt.rcParams['axes.facecolor'] = 'white'
plt.rc('axes',edgecolor='white')
plt.plot(df['Date'].iloc[:-(TEST_SIZE+n_steps_out-1)], df['Price'].iloc[:-(TEST_SIZE+n_steps_out-1)], color='black', lw=2)
plt.plot(df['Date'].iloc[-(TEST_SIZE+n_steps_out-1):], df['Price'].iloc[-(TEST_SIZE+n_steps_out-1):], color='blue', lw=2)
plt.plot(df['Date'].iloc[-(TEST_SIZE+n_steps_out-1):], y_price_test_pred_model5, color='red', lw=2)
plt.title('Biểu đồ thể hiện giá vàng phán đoán của mô hình LSTM', fontsize=15)
plt.xlabel('Ngày', fontsize=12)
plt.ylabel('Giá vàng', fontsize=12)
plt.legend(['Training Data', 'Actual Test Data', 'Predicted Test Data'], loc='upper left', prop={'size': 15})
plt.grid(color='white')
plt.show()

In [ ]:
print('MAPE: {}'.format(mean_absolute_percentage_error(df['Price'].iloc[-(TEST_SIZE+n_steps_out-1):].values, y_price_test_pred_model5)))
print('Accuracy: {}'.format((1 - mean_absolute_percentage_error(df['Price'].iloc[-(TEST_SIZE+n_steps_out-1):].values, y_price_test_pred_model5))*100))

- Dự đoán 50 quan sát thời gian trong tương lai

In [ ]:
n_iters = 3
n_steps_data = df[columns].iloc[-(n_steps_in + n_steps_out * n_iters): -(n_steps_out * n_iters), :]
n_steps_prediction_model5 = msp_data_model.multi_step_forecast(model5, n_steps_data, n_iters)

In [ ]:
plt.figure(figsize=(15, 6), dpi=150)
plt.rcParams['axes.facecolor'] = 'white'
plt.rc('axes',edgecolor='white')
plt.plot(df['Date'].iloc[-(n_steps_in + n_steps_out * n_iters): -(n_steps_out * n_iters)], df['Price'].iloc[-(n_steps_in + n_steps_out * n_iters): -(n_steps_out * n_iters)], color='black', lw=2)
plt.plot(df['Date'].iloc[-(n_steps_out * n_iters):], df['Price'].iloc[-(n_steps_out * n_iters):], color='blue', lw=2)
plt.plot(df['Date'].iloc[-(n_steps_out * n_iters):], n_steps_prediction_model5, color='red', lw=2)
plt.title(f'Biểu đồ thể hiện phán đoán của mô hình LSTM {n_steps_out * n_iters} ngày liên tiếp trong tương lai', fontsize=15)
plt.xlabel('Ngày', fontsize=12)
plt.ylabel('Giá vàng', fontsize=12)
plt.legend(['Actual data', 'Unseen data', 'Predicted Data'], loc='upper left', prop={'size': 15})
plt.grid(color='white')
plt.show()

- Từ biểu đồ cho thấy khả năng dự đoán xa của mô hình khá kém do dữ liệu là ngẫu nhiên không có tính chu kì hay tính dừng 

#### 3.2. Dự đoán multistep bằng mô hình GRU


In [ ]:
def gru_multi_step_model():
    input = Input(shape=X_train[0].shape)
    x = GRU(units=64, return_sequences=False)(input)
    x = Dropout(0.1)(x)
    x = Dense(32, activation='relu')(x)
    output = Dense(n_steps_out*len(columns))(x)

    model = Model(inputs=input, outputs=[output])
    model.compile(loss='mean_squared_error', optimizer='adam')
    model.summary()
    
    return model

In [ ]:
model6 = gru_multi_step_model()

In [ ]:
history6 = model6.fit(X_train, y_train.reshape(-1, n_steps_out*len(columns)), epochs=50, batch_size=128, validation_split=0.1, verbose=1)

In [ ]:
result6 = model6.evaluate(X_test, y_test.reshape(-1, n_steps_out*len(columns)))
y_test_pred_model6 = model6.predict(X_test)

In [ ]:
y_test_pred_model6 = y_test_pred_model6.reshape(-1, n_steps_out, len(columns))
iterations = TEST_SIZE // n_steps_out + 1
remain = TEST_SIZE % n_steps_out

y_price_test_pred_model6 = []

for i in range(iterations):
    y_price_test_pred_model6 += list(msp_data_model.scaler.inverse_transform(y_test_pred_model6[i*n_steps_out]).T[0])
    
if remain != 0:
    y_price_test_pred_model6 += list(msp_data_model.scaler.inverse_transform(y_test_pred_model6[-1]).T[0][-remain+1:])

In [ ]:
plt.figure(figsize=(15, 6), dpi=150)
plt.rcParams['axes.facecolor'] = 'white'
plt.rc('axes',edgecolor='white')
plt.plot(df['Date'].iloc[:-(TEST_SIZE+n_steps_out-1)], df['Price'].iloc[:-(TEST_SIZE+n_steps_out-1)], color='black', lw=2)
plt.plot(df['Date'].iloc[-(TEST_SIZE+n_steps_out-1):], df['Price'].iloc[-(TEST_SIZE+n_steps_out-1):], color='blue', lw=2)
plt.plot(df['Date'].iloc[-(TEST_SIZE+n_steps_out-1):], y_price_test_pred_model6, color='red', lw=2)
plt.title('Biểu đồ thể hiện giá vàng phán đoán của mô hình GRU', fontsize=15)
plt.xlabel('Ngày', fontsize=12)
plt.ylabel('Giá vàng', fontsize=12)
plt.legend(['Training Data', 'Actual Test Data', 'Predicted Test Data'], loc='upper left', prop={'size': 15})
plt.grid(color='white')
plt.show()

In [ ]:
print('MAPE: {}'.format(mean_absolute_percentage_error(df['Price'].iloc[-(TEST_SIZE+n_steps_out-1):].values, y_price_test_pred_model6)))
print('Accuracy: {}'.format((1 - mean_absolute_percentage_error(df['Price'].iloc[-(TEST_SIZE+n_steps_out-1):].values, y_price_test_pred_model6))*100))

In [ ]:
n_steps_prediction_model6 = msp_data_model.multi_step_forecast(model6, n_steps_data, n_iters)

In [ ]:
plt.figure(figsize=(15, 6), dpi=150)
plt.rcParams['axes.facecolor'] = 'white'
plt.rc('axes',edgecolor='white')
plt.plot(df['Date'].iloc[-(n_steps_in + n_steps_out * n_iters): -(n_steps_out * n_iters)], df['Price'].iloc[-(n_steps_in + n_steps_out * n_iters): -(n_steps_out * n_iters)], color='black', lw=2)
plt.plot(df['Date'].iloc[-(n_steps_out * n_iters):], df['Price'].iloc[-(n_steps_out * n_iters):], color='blue', lw=2)
plt.plot(df['Date'].iloc[-(n_steps_out * n_iters):], n_steps_prediction_model6, color='red', lw=2)
plt.title(f'Biểu đồ thể hiện phán đoán của mô hình GRU {n_steps_out * n_iters} ngày liên tiếp trong tương lai', fontsize=15)
plt.xlabel('Ngày', fontsize=12)
plt.ylabel('Giá vàng', fontsize=12)
plt.legend(['Actual data', 'Unseen data', 'Predicted Data'], loc='upper left', prop={'size': 15})
plt.grid(color='white')
plt.show()

#### 3.3. Dự đoán multistep bằng mô hình BiLSTM

In [ ]:
def bilstm_multi_step_model():
    input = Input(shape=X_train[0].shape)
    x = Bidirectional(LSTM(units=64, return_sequences=False))(input)
    x = Dropout(0.1)(x)
    x = Dense(32, activation='relu')(x)
    output = Dense(n_steps_out*len(columns))(x)

    model = Model(inputs=input, outputs=[output])
    model.compile(loss='mean_squared_error', optimizer='adam')
    model.summary()
    
    return model

In [ ]:
model7 = bilstm_multi_step_model()

In [ ]:
history7 = model7.fit(X_train, y_train.reshape(-1, n_steps_out*len(columns)), epochs=50, batch_size=128, validation_split=0.1, verbose=1)

In [ ]:
result7 = model7.evaluate(X_test, y_test.reshape(-1, n_steps_out*len(columns)))
y_test_pred_model7 = model7.predict(X_test)

In [ ]:
y_test_pred_model7 = y_test_pred_model7.reshape(-1, n_steps_out, len(columns))
iterations = TEST_SIZE // n_steps_out + 1
remain = TEST_SIZE % n_steps_out

y_price_test_pred_model7 = []

for i in range(iterations):
    y_price_test_pred_model7 += list(msp_data_model.scaler.inverse_transform(y_test_pred_model7[i*n_steps_out]).T[0])
    
if remain != 0:
    y_price_test_pred_model7 += list(msp_data_model.scaler.inverse_transform(y_test_pred_model7[-1]).T[0][-remain+1:])

In [ ]:
plt.figure(figsize=(15, 6), dpi=150)
plt.rcParams['axes.facecolor'] = 'white'
plt.rc('axes',edgecolor='white')
plt.plot(df['Date'].iloc[:-(TEST_SIZE+n_steps_out-1)], df['Price'].iloc[:-(TEST_SIZE+n_steps_out-1)], color='black', lw=2)
plt.plot(df['Date'].iloc[-(TEST_SIZE+n_steps_out-1):], df['Price'].iloc[-(TEST_SIZE+n_steps_out-1):], color='blue', lw=2)
plt.plot(df['Date'].iloc[-(TEST_SIZE+n_steps_out-1):], y_price_test_pred_model7, color='red', lw=2)
plt.title('Biểu đồ thể hiện giá vàng phán đoán của mô hình BiLSTM', fontsize=15)
plt.xlabel('Ngày', fontsize=12)
plt.ylabel('Giá vàng', fontsize=12)
plt.legend(['Training Data', 'Actual Test Data', 'Predicted Test Data'], loc='upper left', prop={'size': 15})
plt.grid(color='white')
plt.show()

In [ ]:
print('MAPE: {}'.format(mean_absolute_percentage_error(df['Price'].iloc[-(TEST_SIZE+n_steps_out-1):].values, y_price_test_pred_model7)))
print('Accuracy: {}'.format((1 - mean_absolute_percentage_error(df['Price'].iloc[-(TEST_SIZE+n_steps_out-1):].values, y_price_test_pred_model7))*100))

In [ ]:
n_steps_prediction_model7 = msp_data_model.multi_step_forecast(model7, n_steps_data, n_iters)

In [ ]:
plt.figure(figsize=(15, 6), dpi=150)
plt.rcParams['axes.facecolor'] = 'white'
plt.rc('axes',edgecolor='white')
plt.plot(df['Date'].iloc[-(n_steps_in + n_steps_out * n_iters): -(n_steps_out * n_iters)], df['Price'].iloc[-(n_steps_in + n_steps_out * n_iters): -(n_steps_out * n_iters)], color='black', lw=2)
plt.plot(df['Date'].iloc[-(n_steps_out * n_iters):], df['Price'].iloc[-(n_steps_out * n_iters):], color='blue', lw=2)
plt.plot(df['Date'].iloc[-(n_steps_out * n_iters):], n_steps_prediction_model7, color='red', lw=2)
plt.title(f'Biểu đồ thể hiện phán đoán của mô hình BiLSTM {n_steps_out * n_iters} ngày liên tiếp trong tương lai', fontsize=15)
plt.xlabel('Ngày', fontsize=12)
plt.ylabel('Giá vàng', fontsize=12)
plt.legend(['Actual data', 'Unseen data', 'Predicted Data'], loc='upper left', prop={'size': 15})
plt.grid(color='white')
plt.show()

#### 3.4. Dự đoán multistep bằng mô hình CNN

In [ ]:
def cnn_multi_step_model():
    input = Input(shape=X_train[0].shape)
    x = Conv1D(filters=64, kernel_size=3)(input)
    x = MaxPooling1D(pool_size=3)(x)
    x = Flatten()(x)
    x = Dense(units=64, activation='relu')(x)
    x = Dropout(0.1)(x)
    output = Dense(n_steps_out*len(columns))(x)
    
    model = Model(inputs=input, outputs=[output])
    model.compile(loss='mean_squared_error', optimizer='adam')
    model.summary()
    
    return model

In [ ]:
model8 = cnn_multi_step_model()

In [ ]:
history8 = model8.fit(X_train, y_train.reshape(-1, n_steps_out*len(columns)), epochs=50, batch_size=128, validation_split=0.1, verbose=1)

In [ ]:
result8 = model8.evaluate(X_test, y_test.reshape(-1, n_steps_out*len(columns)))
y_test_pred_model8 = model8.predict(X_test)

In [ ]:
y_test_pred_model8 = y_test_pred_model8.reshape(-1, n_steps_out, len(columns))
iterations = TEST_SIZE // n_steps_out + 1
remain = TEST_SIZE % n_steps_out

y_price_test_pred_model8 = []

for i in range(iterations):
    y_price_test_pred_model8 += list(msp_data_model.scaler.inverse_transform(y_test_pred_model8[i*n_steps_out]).T[0])
    
if remain != 0:
    y_price_test_pred_model8 += list(msp_data_model.scaler.inverse_transform(y_test_pred_model8[-1]).T[0][-remain+1:])

In [ ]:
plt.figure(figsize=(15, 6), dpi=150)
plt.rcParams['axes.facecolor'] = 'white'
plt.rc('axes',edgecolor='white')
plt.plot(df['Date'].iloc[:-(TEST_SIZE+n_steps_out-1)], df['Price'].iloc[:-(TEST_SIZE+n_steps_out-1)], color='black', lw=2)
plt.plot(df['Date'].iloc[-(TEST_SIZE+n_steps_out-1):], df['Price'].iloc[-(TEST_SIZE+n_steps_out-1):], color='blue', lw=2)
plt.plot(df['Date'].iloc[-(TEST_SIZE+n_steps_out-1):], y_price_test_pred_model8, color='red', lw=2)
plt.title('Biểu đồ thể hiện giá vàng phán đoán của mô hình BiLSTM', fontsize=15)
plt.xlabel('Ngày', fontsize=12)
plt.ylabel('Giá vàng', fontsize=12)
plt.legend(['Training Data', 'Actual Test Data', 'Predicted Test Data'], loc='upper left', prop={'size': 15})
plt.grid(color='white')
plt.show()

In [ ]:
print('MAPE: {}'.format(mean_absolute_percentage_error(df['Price'].iloc[-(TEST_SIZE+n_steps_out-1):].values, y_price_test_pred_model8)))
print('Accuracy: {}'.format((1 - mean_absolute_percentage_error(df['Price'].iloc[-(TEST_SIZE+n_steps_out-1):].values, y_price_test_pred_model8))*100))

In [ ]:
n_steps_prediction_model8 = msp_data_model.multi_step_forecast(model8, n_steps_data, n_iters)

In [ ]:
plt.figure(figsize=(15, 6), dpi=150)
plt.rcParams['axes.facecolor'] = 'white'
plt.rc('axes',edgecolor='white')
plt.plot(df['Date'].iloc[-(n_steps_in + n_steps_out * n_iters): -(n_steps_out * n_iters)], df['Price'].iloc[-(n_steps_in + n_steps_out * n_iters): -(n_steps_out * n_iters)], color='black', lw=2)
plt.plot(df['Date'].iloc[-(n_steps_out * n_iters):], df['Price'].iloc[-(n_steps_out * n_iters):], color='blue', lw=2)
plt.plot(df['Date'].iloc[-(n_steps_out * n_iters):], n_steps_prediction_model8, color='red', lw=2)
plt.title(f'Biểu đồ thể hiện phán đoán của mô hình BiLSTM {n_steps_out * n_iters} ngày liên tiếp trong tương lai', fontsize=15)
plt.xlabel('Ngày', fontsize=12)
plt.ylabel('Giá vàng', fontsize=12)
plt.legend(['Actual data', 'Unseen data', 'Predicted Data'], loc='upper left', prop={'size': 15})
plt.grid(color='white')
plt.show()